In [340]:
import numpy as np 
import random
import bisect
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import math

In [232]:
def pretty_matrix_print(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            print(round(matrix[i][j], 5), end=" ")
        print("")

In [2]:
def readJSSP(path):
    with open(path, 'r') as f:
        lines =  f.readlines()
        jobs = [[(int(machine), int(cost)) for machine, cost in zip(line.split()[0::2], line.split()[1::2])]
                for line in lines[1:] if line.strip()]
        
        return jobs

In [12]:
def calcMakespan(jobs, schedule):
    n = len(jobs)
    m = len(jobs[0])
    
    job_end = np.zeros(n, dtype=np.int) # end of prev. tasks for each job 
    machine_end = np.zeros(m, dtype=np.int) # end of prev. task for each machine
    next_task = np.zeros(n, dtype=np.int) # next task for each job
    
    
    for job in schedule:
        machine, cost = jobs[job][next_task[job]]
        
        job_start = max(machine_end[machine], job_end[job])
        
        job_end[job] = job_start + cost
        machine_end[machine] = job_start + cost
        
        next_task[job] += 1
        
    return max(machine_end)

In [13]:
def convert_tour_to_schedule(m, tour):
    return list(map(lambda x: x // m, tour))

In [26]:
def convert_schedule_to_tour(n, m, schedule):
    op_counter = [0 for _ in range(n)]
    tour = []
    #print(schedule)
    for job in schedule:
        #print(job*m + op_counter[job])
        tour.append(job*m + op_counter[job])
        op_counter[job] += 1
        
    return tour

In [29]:
def randomSolution(jobs, num_iter):
    n, m = len(jobs), len(jobs[0])
    best_tour = []
    best_makespan = 999999
    
    
    for _ in range(num_iter):
        solution = [j for j in range(n) for i in range(m)]
        random.shuffle(solution)
        makespan = calcMakespan(jobs, solution)
        
        # convert schedule to tour
        tour = convert_schedule_to_tour(n, m, solution)
        if(makespan < best_makespan):
            best_tour = tour
            best_makespan = makespan
    return best_tour, best_makespan

In [277]:
def get_probability(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta):
    n, m = len(jobs), len(jobs[0])
    curr_operation = tour[-1]
    
    p = []
    for i in next_set:
        phero = pheromones[curr_operation][i]
        
        machine, cost = jobs[i // m][i % m]
        job_compl = max(machine_complitions[machine], job_complitions[i // m]) + cost
        p.append(phero * ((1 / cost) ** beta))
    
#    p = [pheromones[curr_operation][next_set[i]] * (1 / jobs[next_set[i] // m][next_set[i] % m][1]) ** beta 
#         for i in range(len(next_set))]
    s = sum(p)
    p = [x / s for x in p]
    return p

In [354]:
def get_probability_on_first_op(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta):
    n, m = len(jobs), len(jobs[0])
    
    p = [pheromones[next_set[i]] * ((1 / jobs[next_set[i] // m][next_set[i] % m][1]) ** beta)
         for i in range(len(next_set))]
    s = sum(p)
    p = [x / s for x in p]
    return p

In [365]:
def biased_exploitation(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta, get_probability=get_probability):
    p = get_probability(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta)
    prob_cumm = 0
    q = random.random()
    s=0
    #print(p)
    for i, x in enumerate(p):        
        prob_cumm += x
        if(q < prob_cumm):
            s = i
            break
    #print(s) 
    return next_set[min(s, len(next_set) - 1)]

In [280]:
def exploitation(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta, get_probability=get_probability):
    p = get_probability(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta)
    #print(len(tour), next_set)
    return next_set[p.index(max(p))]

In [281]:
def choose_next_operation(job, job_complitions, machine_complitions, pheromones, tour, next_set, beta, q0, get_probability=get_probability):
    q = random.random()
    next_opertaion = -1
    if(q < q0):
        next_opertaion = exploitation(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta, get_probability)
    else:
        next_opertaion = biased_exploitation(jobs, job_complitions, machine_complitions, pheromones, tour, next_set, beta, get_probability)
    return next_opertaion

In [412]:
def optimize(jobs, beta, q0, rho, alpha, num_iter, tau_0_solution=-1, tau_0_tour=[]):
    n, m = len(jobs), len(jobs[0])
    
    # initialization
    if(tau_0_solution != -1):
        best_global_tour, best_global_time = tau_0_tour, tau_0_solution
    else:
        best_global_tour, best_global_time = randomSolution(jobs, 100)
    tau_0 = 1 /  best_global_time # Not great, not terrible 
    print(best_global_time)
    pheromones = [[tau_0 for i in range(n * m)] for j in range(n * m)]
    pheromones_initial = [tau_0 for i in range(n * m)]
    num_ant = n 
    #pretty_matrix_print(pheromones)
    history = []
    #best_global_time = 9999999
    #best_global_tour = []
    # this is phase in which ants build their tours 
    for it in tqdm(range(num_iter)):
        first_iteration = it == 0
        # visited opertaion on each job for each ant
        visited = [[[0 for j in range(m)] for i in range(n)] for _ in range(num_ant)] 
        next_set = [[i * m for i in range(n)] for j in range(num_ant)] # first operation of each job
        time = [0 for i in range(num_ant)]
        tour = [[] for i in range(num_ant)]
        current_op = [-1 for i in range(num_ant)]
        job_complitions = [[0 for i in range(n)] for _ in range(num_ant)]
        machine_complitions = [[0 for i in range(m)] for _ in range(num_ant)]
        for i in range(n * m):
            for k in range(num_ant):
                if(first_iteration and current_op[k] == -1):
                    visited[k][k][0] = 1
                    # k*m - first operation of k-th job
                    next_set[k].append(k * m + 1)
                    next_set[k].remove(k * m)
                    tour[k].append(k * m)
                else:
                    if(current_op[k] == -1):
                        get_pr_function = get_probability_on_first_op
                        phero_matrix = pheromones_initial
                    else:
                        get_pr_function = get_probability
                        phero_matrix = pheromones

                    if(len(next_set) == 1):
                        next_operation = next_set[-1]
                    else:
                        next_operation = choose_next_operation(jobs, job_complitions[k], machine_complitions[k], phero_matrix, tour[k], next_set[k], beta, q0, get_pr_function)
                    machine_complitions[k][jobs[next_operation // m][next_operation % m][0]] += jobs[next_operation // m][next_operation % m][1]
                    job_complitions[k][next_operation // m] += jobs[next_operation // m][next_operation % m][1]
                    tour[k].append(next_operation)
                    visited[k][next_operation // m][next_operation % m] = 1
                    next_set[k].remove(next_operation)
                    if((next_operation + 1) % m != 0):
                        next_set[k].append(next_operation+1)

            # local updating

            for k in range(num_ant):
                if(not first_iteration):
                    pheromones[current_op[k]][tour[k][-1]] = (1 - rho) * pheromones[current_op[k]][tour[k][-1]] \
                                                            + rho * tau_0
                current_op[k] = tour[k][-1]

        # update global results
        for k in range(num_ant):
            time[k] = calcMakespan(jobs, convert_tour_to_schedule(m, tour[k]))
        #print("it: ", it)
        
        best_time = min(time)
        best_tour = tour[time.index(best_time)]
        #print(time)
        #print(tour)
        if(best_time < best_global_time):
            best_global_time = best_time
            best_global_tour = best_tour
        #print(best_global_time)
        #print(best_global_tour)
        old_phero = [[pheromones[i][j] for j in range(len(pheromones[i]))] for i in range(len(pheromones))]
        # global update pheromones
        for i in range(len(pheromones)):
            for j in range(len(pheromones[i])):
                pheromones[i][j] *= (1 - alpha)
        
        
        pheromones_initial[best_global_tour[0]] += 10 * alpha / best_global_time
        for i in range(1, len(best_global_tour)):
            pheromones[best_global_tour[i-1]][best_global_tour[i]] += 10 * alpha / best_global_time
        
        
        #pretty_matrix_print([[pheromones[i][j] - old_phero[i][j] for j in range(len(pheromones[i]))] for i in range(len(pheromones))])
        
        
        history.append({
            "it": it,
            "best_time": best_time,
            "best_tour": best_tour
        })
        
    return best_global_time, best_global_tour, history

In [ ]:
jobs = readJSSP("./instanses/la36")
beta = 1
alpha = 0.2
rho = 0.01
q0 = 0.9
num_iter = 300


best_time, best_tour, history = optimize(jobs, beta, q0, rho, alpha, num_iter, tau_0_solution, tau_0_tour)

best_time

1599


In [ ]:
tau_0_tour = best_tour
tau_0_solution = best_time

In [ ]:
x = [it["it"] for it in history]
y = [it["best_time"] for it in history]

plt.figure(figsize=(16, 9))
plt.plot(x, y)

In [380]:
[(x[i], y[i]) for i in range(len(history[:100]))]

[(0, 2387),
 (1, 2823),
 (2, 2650),
 (3, 2691),
 (4, 2499),
 (5, 2507),
 (6, 2726),
 (7, 2372),
 (8, 2070),
 (9, 2225),
 (10, 2194),
 (11, 2472),
 (12, 2515),
 (13, 2634),
 (14, 2609),
 (15, 2421),
 (16, 2290),
 (17, 2631),
 (18, 2738),
 (19, 2487),
 (20, 2609),
 (21, 2677),
 (22, 2803),
 (23, 2258),
 (24, 2757),
 (25, 2479),
 (26, 2739),
 (27, 2518),
 (28, 2722),
 (29, 2618),
 (30, 2837),
 (31, 2734),
 (32, 2595),
 (33, 2497),
 (34, 2675),
 (35, 2772),
 (36, 2411),
 (37, 2625),
 (38, 2766),
 (39, 2722),
 (40, 2857),
 (41, 2832),
 (42, 2881),
 (43, 2902),
 (44, 2931),
 (45, 2672),
 (46, 2659),
 (47, 2763),
 (48, 2914),
 (49, 2931),
 (50, 2672),
 (51, 2807),
 (52, 2783),
 (53, 2957),
 (54, 2807),
 (55, 2747),
 (56, 2697),
 (57, 2855),
 (58, 2840),
 (59, 2757),
 (60, 2814),
 (61, 2833),
 (62, 3029),
 (63, 2967),
 (64, 2942),
 (65, 2807),
 (66, 3059),
 (67, 2942),
 (68, 2807),
 (69, 2732),
 (70, 3059),
 (71, 2847),
 (72, 2847),
 (73, 2942),
 (74, 2805),
 (75, 2942),
 (76, 2644),
 (77, 284